# 处理地点信息，新增临近节点

In [80]:
import pandas as pd
import numpy as np
import urllib
from datetime import datetime

In [4]:
path = 'C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/street_dataset.csv'
data = pd.read_csv(path)
data.head()

,日期,district,street,经停病例数,患者数量,全市患者数量,所在区患者数量,封控与否
0,2021/12/23 0:00,津南区,永胜道,31,0,0,0,0
1,2021/12/24 0:00,津南区,永胜道,42,0,0,0,0
2,2021/12/25 0:00,津南区,永胜道,36,0,0,0,0
3,2021/12/26 0:00,津南区,永胜道,46,0,0,0,0
4,2021/12/27 0:00,津南区,永胜道,73,0,0,0,0


In [163]:
last_street=""
last_district=""
area_address_message=pd.DataFrame(columns=['district','street','经度','纬度','nerghbor1','nerghbor2','nerghbor3'])

In [164]:
index=0
for i in range(data.shape[0]):
    if data.iloc[i,2]!=last_street:
        last_street=data.iloc[i,2]
        area_address_message.loc[index,'district']=data.iloc[i,1]
        area_address_message.loc[index,'street']=data.iloc[i,2]
        index+=1

In [86]:
address=""
API_key = "38spFG65KGTl8NMvOmfyQLN53EGYSoZG"  # 使用百度地图地理编码API所需的key,这里我申请的是个人key,每日只能查询5000条数据
city = "天津"  # 默认对天津的地点进行地理编码和逆编码
city = urllib.parse.quote(city)  # 这里要注意,按照标准,URL只允许一部分ASCII字符,其他字符（如汉字）是不符合标准的，此时就要进行编码,下方的address同样需要

In [87]:
for index,row in area_address_message.iterrows():
    if address is not np.NaN:
        address = row['district']+row['street']
        address = "天津市" + address
        address = urllib.parse.quote(address)
        geo_url = "https://api.map.baidu.com/geocoding/v3/?ak={}&address={}&city={}&output=json".format(API_key, address, city)

        req = urllib.request.Request(geo_url)  # 将url对象转化为request对象
        try:
            ret = urllib.request.urlopen(req)  # 发送请求，打开url，也可以直接传入url
            result = ret.read().decode('utf-8')
            result = eval(result)
            if result['status'] == 0:  # 有时候API会有一些没法返回正常的查询结果,status为0是正常返回结果
                location = result['result']['location']
                lng, lat = location['lng'], location['lat']  # 地点对应经纬度

                regeo_url = "https://api.map.baidu.com/reverse_geocoding/v3/?ak={}&location={},{}&output=json&extensions_poi=1&extensions_road=true".format(API_key, lat, lng)
                req = urllib.request.Request(regeo_url)  # 将url对象转化为request对象
                try:
                    ret = urllib.request.urlopen(req)  # 发送请求，打开url，也可以直接传入url
                    result = ret.read().decode('utf-8')
                    result = eval(result)
                    if result['status'] == 0:
                        # formatted_address = result['result']['formatted_address']  # 完整地址
                        # district = result['result']['addressComponent']['district']  # 所在区
                        # street = result['result']['addressComponent']['street']  # 所在街道
                        # street_number = result['result']['addressComponent']['street_number']  # 所在街道门牌号
                        # business = result['result']['business']  # 所在商圈
                        # place_around = result['result']['pois']  # 附近地点，list类型
                        # road1, road2, road3 = '', '', ''
                        # if len(result['result']['roads']) == 3:
                        #     road1 = result['result']['roads'][0]['name']  # 附近的路
                        #     road2 = result['result']['roads'][1]['name']
                        #     road3 = result['result']['roads'][2]['name']
                        # elif len(result['result']['roads']) == 2:
                        #     road1 = result['result']['roads'][0]['name']  # 附近的路
                        #     road2 = result['result']['roads'][1]['name']
                        # elif len(result['result']['roads']) == 1:
                        #     road1 = result['result']['roads'][0]['name']  # 附近的路

                        # if formatted_address:  # 主要是防止有些地址返回的是空集
                        #     data.loc[index, 'formatted_address'] = formatted_address
                        # if district:
                        #     data.loc[index, 'district'] = district
                        # if street:
                        #     data.loc[index, 'street'] = street
                        # if street_number:
                        #     data.loc[index, 'street_number'] = street_number
                        if lng:
                            area_address_message.loc[index,'经度'] = lng
                        if lat:
                            area_address_message.loc[index,'纬度'] = lat
                        # if business:
                        #     data.loc[index, 'business'] = business
                        # if place_around:
                        #     data.loc[index, 'place_around'] = index  # 这里只存索引，另外拿一张表存附近地区信息，根据索引查找
                        #     place.append(place_around)
                        # if road1:
                        #     data.loc[index, 'road1'] = road1
                        # if road2:
                        #     data.loc[index, 'road2'] = road2
                        # if road3:
                        #     data.loc[index, 'road3'] = road3
                except urllib.error.URLError as e:
                    print("open url error")
        except urllib.error.URLError as e:
            print("url error")
        
        

open url error
url error


In [89]:
# 根据经纬度计算距离
from math import radians, cos, sin, asin, sqrt
import heapq
#lng1,lat1,lng2,lat2 = (120.12802999999997,30.28708,115.86572000000001,28.7427)
def geodistance(lng1,lat1,lng2,lat2):
    lng1, lat1, lng2, lat2 = map(radians, [float(lng1), float(lat1), float(lng2), float(lat2)]) # 经纬度转换成弧度
    dlon=lng2-lng1
    dlat=lat2-lat1
    a=sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    distance=2*asin(sqrt(a))*6371*1000 # 地球平均半径，6371km
    distance=round(distance/1000,3)
    return distance

In [112]:
minindexs=[]
for i in range(area_address_message.shape[0]):
    lng1=area_address_message.iloc[i,2]
    lat1=area_address_message.iloc[i,3]
    distances=[]
    for j in range(area_address_message.shape[0]):
        lng2=area_address_message.iloc[j,2]
        lat2=area_address_message.iloc[j,3]
        distances.append(geodistance(lng1,lat1,lng2,lat2))
    # print(distances)
    min_number = heapq.nsmallest(4, distances) 
    # print(min_number)
    min_index = []
    for t in min_number:
        if t==0:
            index = distances.index(t)
            distances[index] = 1000000000
            continue
        if t>=10:
            break
        index = distances.index(t)
        # print(index)
        min_index.append(index)
        distances[index] = 1000000000
    # print(min_index)
    minindexs.append(min_index)

In [162]:
minindexs[60]

[359, 16]

In [168]:
area_address_message.iloc[59,:]

district           静海区
street       S116(津文线)
经度                 NaN
纬度                 NaN
nerghbor1          NaN
nerghbor2          NaN
nerghbor3          NaN
Name: 59, dtype: object

In [166]:
# 用 经停病例数，数据较多
for i in range(len(minindexs)):
    for j in range(len(minindexs[i])):
        # 主要因为有重复距离相同地区，使len>3
        if j>=3:
            break
        # print('j ',j)
        idx=minindexs[i][j]
        # print('idx ',idx)
        area_address_message.iloc[i,j+4]=idx


In [127]:
data.drop(['封控与否'],axis=1,inplace=True)

In [129]:
data = pd.concat([data, pd.DataFrame(columns=['邻近地区1经停病例数','邻近地区2经停病例数','邻近地区3经停病例数','封控与否'])], sort=False)

In [206]:
# 用经停病例
# area_address_message.fillna(0)
# data.fillna(0)
index=0
start_1=area_address_message.iloc[index,4]*41
start_2=area_address_message.iloc[index,5]*41
start_3=area_address_message.iloc[index,6]*41
for i in range(data.shape[0]):
    # print(start_1,' ',start_2,' ',start_3)
    if data.iloc[i,2]==area_address_message.iloc[index,1]:
        # print('jinlai')
        if start_1>=0:
            # print('jinlai2  ',i)
            start_1+=1
            data.iloc[i,7]=data.iloc[start_1,3] 
            # print(data.iloc[i,7])
        if start_2>=0:
            start_2+=1
            data.iloc[i,8]=data.iloc[start_2,3]
        if start_3>=0:
            start_3+=1
            data.iloc[i,9]=data.iloc[start_3,3]
    
    else:
        index+=1
        start_1=area_address_message.iloc[index,4]*41
        start_2=area_address_message.iloc[index,5]*41
        start_3=area_address_message.iloc[index,6]*41
        # print('index',index)
        # print('start_3',start_3)
        if start_1>=0:
            data.iloc[i,7]=data.iloc[start_1,3] 
        if start_2>=0:
            data.iloc[i,8]=data.iloc[start_2,3]
        if start_3>=0:
            data.iloc[i,9]=data.iloc[start_3,3]
    # print(data.iloc[i,:])

In [207]:
outputpath='C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/dataset_palce.csv'
data.to_csv(outputpath,sep=',',index=False,header=True,encoding='utf_8_sig')  # 保存结果

# 处理label

In [210]:
import openpyxl
# path = 'C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/dataset_place.csv'
# data = pd.read_csv(path)
# data.head()
path = 'C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/原始数据/事件数据.xlsx'
event = pd.read_excel(path)
event.head()


In [220]:
event.iloc[1149,:]

日期          2022-02-20 00:00:00
场所                    河北区所有培训机构
事件                           开放
district                    NaN
street                      NaN
Name: 1149, dtype: object

In [219]:
event.drop(['起始时间','来源'],axis=1,inplace=True)
event = pd.concat([event, pd.DataFrame(columns=['district','street'])], sort=False)
event.head()

1151

In [222]:
index=0
event_new=pd.DataFrame(columns=['日期','场所','事件','district','street'])
for i in range(len(event)):
    if "所有" not in event.iloc[i,1] and "全部" not in event.iloc[i,1]:
        # print(event_data.iloc[i,2])
        event_new.loc[index,:]=event.iloc[i,:]
        index+=1
event_new.head()

,日期,场所,事件,district,street
0,2022-01-08 00:00:00,辛庄镇碧水世纪园,划定封控区,NaN,NaN
1,2022-01-08 00:00:00,华远波士顿上东轩,划定封控区,NaN,NaN
2,2022-01-08 00:00:00,鑫昱花园,划定封控区,NaN,NaN
3,2022-01-08 00:00:00,林锦花园,划定封控区,NaN,NaN
4,2022-01-08 00:00:00,义佳花园,划定封控区,NaN,NaN


In [215]:
def regular_place(data):
    address=""
    for index,row in data.iterrows():
        if address is not np.NaN:
            address = row['场所']
            address = "天津市" + address
            address = urllib.parse.quote(address)
            geo_url = "https://api.map.baidu.com/geocoding/v3/?ak={}&address={}&city={}&output=json".format(API_key, address, city)

            req = urllib.request.Request(geo_url)  # 将url对象转化为request对象
            try:
                ret = urllib.request.urlopen(req)  # 发送请求，打开url，也可以直接传入url
                result = ret.read().decode('utf-8')
                result = eval(result)
                if result['status'] == 0:  # 有时候API会有一些没法返回正常的查询结果,status为0是正常返回结果
                    location = result['result']['location']
                    lng, lat = location['lng'], location['lat']  # 地点对应经纬度

                    regeo_url = "https://api.map.baidu.com/reverse_geocoding/v3/?ak={}&location={},{}&output=json&extensions_poi=1&extensions_road=true".format(API_key, lat, lng)
                    req = urllib.request.Request(regeo_url)  # 将url对象转化为request对象
                    try:
                        ret = urllib.request.urlopen(req)  # 发送请求，打开url，也可以直接传入url
                        result = ret.read().decode('utf-8')
                        result = eval(result)
                        if result['status'] == 0:
                            formatted_address = result['result']['formatted_address']  # 完整地址
                            district = result['result']['addressComponent']['district']  # 所在区
                            street = result['result']['addressComponent']['street']  # 所在街道
                            # street_number = result['result']['addressComponent']['street_number']  # 所在街道门牌号
                            # business = result['result']['business']  # 所在商圈
                            # place_around = result['result']['pois']  # 附近地点，list类型
                            # road1, road2, road3 = '', '', ''
                            # if len(result['result']['roads']) == 3:
                            #     road1 = result['result']['roads'][0]['name']  # 附近的路
                            #     road2 = result['result']['roads'][1]['name']
                            #     road3 = result['result']['roads'][2]['name']
                            # elif len(result['result']['roads']) == 2:
                            #     road1 = result['result']['roads'][0]['name']  # 附近的路
                            #     road2 = result['result']['roads'][1]['name']
                            # elif len(result['result']['roads']) == 1:
                            #     road1 = result['result']['roads'][0]['name']  # 附近的路

                            # if formatted_address:  # 主要是防止有些地址返回的是空集
                            #     data.loc[index, 'formatted_address'] = formatted_address
                            if district:
                                data.loc[index, 'district'] = district
                            if street:
                                data.loc[index, 'street'] = street
                            # if street_number:
                            #     data.loc[index, 'street_number'] = street_number
                            # if lng:
                            #     area_address_message.loc[index,'经度'] = lng
                            # if lat:
                            #     area_address_message.loc[index,'纬度'] = lat
                            # if business:
                            #     data.loc[index, 'business'] = business
                            # if place_around:
                            #     data.loc[index, 'place_around'] = index  # 这里只存索引，另外拿一张表存附近地区信息，根据索引查找
                            #     place.append(place_around)
                            # if road1:
                            #     data.loc[index, 'road1'] = road1
                            # if road2:
                            #     data.loc[index, 'road2'] = road2
                            # if road3:
                            #     data.loc[index, 'road3'] = road3
                    except urllib.error.URLError as e:
                        print("open url error")
            except urllib.error.URLError as e:
                print("url error")
        
        

In [223]:
regular_place(event_new)
event_new

,日期,场所,事件,district,street
0,2022-01-08 00:00:00,辛庄镇碧水世纪园,划定封控区,津南区,辛柴路桥
1,2022-01-08 00:00:00,华远波士顿上东轩,划定封控区,津南区,永胜道
2,2022-01-08 00:00:00,鑫昱花园,划定封控区,津南区,永红道
3,2022-01-08 00:00:00,林锦花园,划定封控区,津南区,永胜道
4,2022-01-08 00:00:00,义佳花园,划定封控区,津南区,慧通道
...,...,...,...,...,...
1001,2022-02-16 00:00:00,建昌道（育红路为起点）-小红星路-泰兴路-金钟河大街-育红路（建昌道为终点）围合区域,解除防范区管控,河北区,金钟河大街
1002,2022-02-16 00:00:00,方特欢乐世界,恢复运营,滨海新区,中生大道
1003,2022-02-18 00:00:00,天津长客西站汽车站,开站,河西区,宾水道
1004,2022-02-17 00:00:00,天津通莎客运站,恢复运营,河东区,新兆路


In [227]:
event_new.to_csv("C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/事件数据+详细地点.csv")

In [230]:
event_temp=event_new
event_temp


,日期,场所,事件,district,street
0,2022-01-08 00:00:00,辛庄镇碧水世纪园,划定封控区,津南区,辛柴路桥
1,2022-01-08 00:00:00,华远波士顿上东轩,划定封控区,津南区,永胜道
2,2022-01-08 00:00:00,鑫昱花园,划定封控区,津南区,永红道
3,2022-01-08 00:00:00,林锦花园,划定封控区,津南区,永胜道
4,2022-01-08 00:00:00,义佳花园,划定封控区,津南区,慧通道
...,...,...,...,...,...
1001,2022-02-16 00:00:00,建昌道（育红路为起点）-小红星路-泰兴路-金钟河大街-育红路（建昌道为终点）围合区域,解除防范区管控,河北区,金钟河大街
1002,2022-02-16 00:00:00,方特欢乐世界,恢复运营,滨海新区,中生大道
1003,2022-02-18 00:00:00,天津长客西站汽车站,开站,河西区,宾水道
1004,2022-02-17 00:00:00,天津通莎客运站,恢复运营,河东区,新兆路


In [259]:
label={0:['解除防范区管控','解除管控区管控','解除封控区管控','恢复运营','恢复部分运营','开放','开启','开站'],1:['划定防范区','划定管控区','划定封控区','划定低风险地区',
        '划定中风险地区','划定高风险地区','只出不进','区间运营','停运','改道运营','关闭']}
def get_keys(d, value):
    return [k for k,v in d.items() if value in v]

In [258]:
event_temp.iloc[1003,:]

日期          2022-02-18 00:00:00
场所                    天津长客西站汽车站
事件                           开站
district                    河西区
street                      宾水道
Name: 1003, dtype: object

In [261]:
for i in range(len(event_temp)):
    # print(i)
    event_temp.iloc[i,1]=get_keys(label,event_temp.iloc[i,2])[0] 

In [287]:
import time
event_temp.iloc[0,0].strftime('%Y/%m/%d').replace('/0','/')==data.iloc[16,0].split(' ')[0]
# data.iloc[16,0].split(' ')[0]
# time.strftime('%Y_/#%m/%d',event_temp.iloc[0,0])
# time.localtime()

True

In [290]:
for i in range(data.shape[0]):
    # 跟数据排布有关，但是现在只能遍历，复杂度很高
    for j in range(event_temp.shape[0]):
        if data.iloc[i,0].split(' ')[0]==event_temp.iloc[j,0].strftime('%Y/%m/%d').replace('/0','/') and data.iloc[i,2]==event_temp.iloc[j,4]:
            data.iloc[i,10]=event_temp.iloc[j,2]
            break
    

In [293]:
outputpath='C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/dataset_palce_label.csv'
data.to_csv(outputpath,sep=',',index=False,header=True,encoding='utf_8_sig')  # 保存结果

In [299]:

data_temp=data
data_temp=data_temp.fillna(0)
# data_temp
for i in range(len(data)):
    # print(i)
    if(data_temp.iloc[i,10]!=0):
        data_temp.iloc[i,10]=get_keys(label,data_temp.iloc[i,10])[0] 
    

In [303]:
# 遍历，如果是1，则打上7天1
danger=0
flag=0
for i in range(len(data)):
    # print(i)
    if flag:
        data_temp.iloc[i,10]=1
        danger-=1
        if danger==0:
            flag=0
        continue

    if(data_temp.iloc[i,10]==1):
        flag=1
        danger=6

In [305]:
outputpath='C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/dataset_palce_label_0_1.csv'
data_temp.to_csv(outputpath,sep=',',index=False,header=True,encoding='utf_8_sig')  # 保存结果
# data_temp

In [34]:
# 用百度地图读出来的邻居去查，效果不好

path = 'C:/Users/26937/Desktop/封控事件预测/huangtianhao/Prediction-of-Containment-Events/数据/Baidu_Trajectory_Geocoding.csv'
temp = pd.read_csv(path)
neighbor_data=temp[['district','street','经度','纬度','road1','road2','road3']]
neighbor_data=neighbor_data.drop_duplicates()
neighbor_data=neighbor_data.reset_index(drop=True)

In [35]:
neighbor_data

,district,street,经度,纬度,road1,road2,road3
0,南开区,内环山路,117.161653,39.082885,内环山路,翠坪桥,晓月桥
1,河西区,环湖西路,117.190323,39.085910,环湖西路,宾水道,环湖西路辅路
2,河西区,福建路,117.231663,39.112418,福建路,奉化道,闽侯路
3,河西区,马场道,117.213347,39.114443,马场道,金字塔路,樱花路
4,河西区,西横街,117.281894,39.075696,五二新村路,鄱阳路天桥,大沽南路辅路
...,...,...,...,...,...,...,...
666,河西区,大沽南路辅路,117.265601,39.083287,大沽南路辅路,大沽南路,墨江道
667,河西区,东江道,117.255723,39.077663,东江道,浩江南道,崇岩东路
668,红桥区,勤俭道,117.162780,39.179551,光荣道,勤俭道辅路,勤俭道
669,红桥区,丁字沽一号路,117.164164,39.188224,丁字沽一号路,向东道,白酒厂大道


In [74]:
nerghbor_list=[]
for i in range(neighbor_data.shape[0]):
    nerghbor_list.append(neighbor_data.iloc[i,1])

# # neighbor_data.infer_objects().dtypes
# pd.to_numeric(neighbor_data[])
# print(nerghbor_list[0])
# # type('1')\
# help(pd.DataFrame.infer_objects)

In [53]:
type(area_address_message.iloc[4,1])

str

In [73]:
# print('内环山路') 
'内环山路'==nerghbor_list[0]

True

In [78]:
nerghbor_list.index('内环山路')

0

In [79]:
num=0
for i in range(area_address_message.shape[0]):
    if area_address_message.iloc[i,1] in nerghbor_list:
        index=nerghbor_list.index(area_address_message.iloc[i,1])
        area_address_message.iloc[i,2]=neighbor_data.iloc[index,2]
        area_address_message.iloc[i,3]=neighbor_data.iloc[index,3]
        area_address_message.iloc[i,4]=neighbor_data.iloc[index,4]
        area_address_message.iloc[i,5]=neighbor_data.iloc[index,5]
        
area_address_message

# print(num)

,district,street,经度,纬度,nerghbor1,nerghbor2,nerghbor3
0,津南区,永胜道,117.359155,39.03805,永胜道,张满庄路,NaN
1,河北区,东二经路,NaN,NaN,NaN,NaN,NaN
2,南开区,云际道,NaN,NaN,NaN,NaN,NaN
3,西青区,储源道,NaN,NaN,NaN,NaN,NaN
4,西青区,业盛道,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
827,河西区,兴湾道,NaN,NaN,NaN,NaN,NaN
828,东丽区,X438,NaN,NaN,NaN,NaN,NaN
829,滨海新区,X423(洋苏路),NaN,NaN,NaN,NaN,NaN
830,和平区,赤峰道,NaN,NaN,NaN,NaN,NaN
